#**WEEK 5**

This week we'll be working through Chapter 3 of Applied Bioinformatics. We will be running BLAST on the command line after reading week, and this textbook has a really good explanation of how that is implemented. For today's lab, we'll be looking at how sequence alignment works under the hood using the two alignment methods we discussed in class in Week 3: global alignment and local alignment.

First, we have to install BioPython again, since this is a new notebook. You can do that by running the code block below.

In [ ]:
##Install BioPython in Jupyter Notebook
%pip install biopython

Before we start, try having a go at aligning some sequences using the Hamming distance between them.



In [ ]:
seq_1 = "ATCGATCGATCG"
seq_2 = "ATCGATCGATCA"
distance = sum(n1 != n2 for n1, n2 in zip(seq_1, seq_2))

print("The Hamming distance between these two sequences is " + str(distance))

The Hamming distance between these two sequences is 1


Try changing the sequences. What kinds of sequences can be aligned? What sequences can't be aligned?

\---------------------------
You can add your answer in this text box.
\---------------------------

#Part 1: Global Alignment
Recall what you have learned about the Needleman-Wunsch algorithm, which we covered in class about two weeks ago. Well, you covered it in class two weeks ago, I while writing this in December 2023 haven't even finished prepping that lecture yet. Time is weird.

In the lecture, we used the pre-computed version of the algorithm written by the textbook author and implemented in numpy. As you can imagine, BioPython has a handily written function that does this job for us as well. In the previous iteration of BioPython one could specify which algorithm to use; with the new align package the choice of global alignment algorithm is optimised behind the scenes.

In [ ]:
##This code defines two sequences and aligns them using global alignment.
from Bio import Align
aligner = Align.PairwiseAligner()
aligner.mode = 'global'
aligner.match_score = 2
aligner.mismatch_score = -1
alignment = aligner.align("TACTAA", "ATCTG")
for alignment in aligner.align("TACTAA", "ATCTG"):
    print("Score = %.1f:" % alignment.score)
    print(alignment)

Is this the output you expected? Why or why not?

\---------------------------
You can add your answer in this text box.
\---------------------------

Try altering the `match_score` and `mismatch_score` parameters. How does this affect the output? Why do you think that is?

\---------------------------
You can add your answer in this text box.
\---------------------------

#Part 2: Local Alignment
Let's repeat the steps above, but with a local instead of global algorithm.

In [ ]:
##This code defines two sequences and aligns them using global alignment.
from Bio import Align
aligner = Align.PairwiseAligner()
aligner.mode = 'local'
aligner.match_score = 2
aligner.mismatch_score = -1
alignment = aligner.align("TACTAA", "ATCTG")
for alignment in aligner.align("TACTAA", "ATCTG"):
    print("Score = %.1f:" % alignment.score)
    print(alignment)

Score = 6.0:
target            1 A-CT 4
                  0 |-|| 4
query             0 ATCT 4

Score = 6.0:
target            0 TACT 4
                  0 |-|| 4
query             1 T-CT 4



Is this the output you expected? Why or why not?

\---------------------------
You can add your answer in this text box.
\---------------------------

Try altering the `match_score` and `mismatch_score` parameters. How does this affect the output? Why do you think that is?

\---------------------------
You can add your answer in this text box.
\---------------------------

#Part 3: Substitution matricies
Substitution matricies allow us to add weight to our alignment calculations. We've already sort of been doing that with the `match_score` and the `mismatch_score`, but substitution matricies allow us to define these relationships in more detail.

First, let's define a simple substitution matrix: a score of 2 for no change, a score of 0 for a transtition (pyrimidine - pyrimidine or purine - purine) and -1 for a transversion (pyrimidine - purine or vice versa)


In [ ]:
##This code defines a substitution matrix
substitution_matrix = {
    'A': {'A': 2, 'C': -1, 'G': 0, 'T': -1},
    'C': {'A': -1, 'C': 2, 'G': -1, 'T': 0},
    'G': {'A': 0, 'C': -1, 'G': 2, 'T': -1},
    'T': {'A': -1, 'C': 0, 'G': -1, 'T': 2}
}

How many times does each number appear in the matrix? Why?

\---------------------------
You can add your answer in this text box.
\---------------------------

To actually *use* our substitution matrix, we will need to define a function that allows us to pull scores from it. Don't worry too much about how this is done as Python functions are beyond the scope of this course, but feel free to ask Brian annoying questions about how it works. The important thing to know is that when you define a function using `def`, you set **parameters**, which are the pieces of information the function needs to know to run. For this function, which calculates the substitution score between each amino acid, only two parameters are needed: the original base pair and the substituted base pair.

In [ ]:
##This code defines a function that calculates substitution scores and shows a test example
def get_substitution_score(nucleotide1, nucleotide2):
    return substitution_matrix[nucleotide1][nucleotide2]

score = get_substitution_score("A","C")
print(score)

-1


Now we're going to combine the Needleman-Wunsch algorithm (one of our global alignment algorithms defined above) with our substitution matrix to run some alignments. The code in this block is DEFINITELY beyond the scope of this course, but once again feel free to ask Brian annoying questions about it.

In [ ]:
def needleman_wunsch(sequence1, sequence2, substitution_matrix, gap_penalty=-1):
    # Initialize the scoring matrix
    rows = len(sequence1) + 1
    cols = len(sequence2) + 1
    score_matrix = [[0] * cols for _ in range(rows)]

    # Initialize the scoring matrix with gap penalties
    for i in range(1, rows):
        score_matrix[i][0] = score_matrix[i-1][0] + gap_penalty

    for j in range(1, cols):
        score_matrix[0][j] = score_matrix[0][j-1] + gap_penalty

    # Fill in the scoring matrix based on the substitution matrix
    for i in range(1, rows):
        for j in range(1, cols):
            match_score = score_matrix[i-1][j-1] + substitution_matrix[sequence1[i-1]][sequence2[j-1]]
            gap1_score = score_matrix[i-1][j] + gap_penalty
            gap2_score = score_matrix[i][j-1] + gap_penalty

            # Choose the maximum score at the current position
            score_matrix[i][j] = max(match_score, gap1_score, gap2_score)

    # Traceback to find the optimal alignment
    alignment1 = ""
    alignment2 = ""
    i, j = rows - 1, cols - 1

    while i > 0 or j > 0:
        if i > 0 and j > 0 and score_matrix[i][j] == score_matrix[i-1][j-1] + substitution_matrix[sequence1[i-1]][sequence2[j-1]]:
            alignment1 = sequence1[i-1] + alignment1
            alignment2 = sequence2[j-1] + alignment2
            i -= 1
            j -= 1
        elif i > 0 and score_matrix[i][j] == score_matrix[i-1][j] + gap_penalty:
            alignment1 = sequence1[i-1] + alignment1
            alignment2 = "-" + alignment2
            i -= 1
        else:
            alignment1 = "-" + alignment1
            alignment2 = sequence2[j-1] + alignment2
            j -= 1

    return alignment1, alignment2

Which parameters are included in the function? What does each of them mean?

\---------------------------
You can add your answer in this text box.
\---------------------------

Now we have a function and a substitution matrix; let's try running it!

In [ ]:
##This code defines two sequences and runs a Needleman-Wunsch alignment of those sequences using a custom substitution matrix.

seq_1 = "Your DNA sequence"
seq_2 = "Your DNA sequence"

alignment = needleman_wunsch(seq_1, seq_2, substitution_matrix)

print(alignment)

('ATGTGTTAGAT', 'ATGTGTTAG-T')


Have a go at changing the sequences. What do you observe?

\---------------------------
You can add your answer in this text box.
\---------------------------

© Elisabeth Richardson, 2023. Adapted from Applied Bioinformatics by David A. Hendricks under a [CC-by-4.0 license](https://https://creativecommons.org/licenses/by/4.0/) and shared under same.